# Imports

In [12]:
import numpy as np 
import pandas as pd
import json 

import altair as alt
from altair import expr, datum

import plotly.express as px

# Data Reading

In [3]:
data=pd.read_excel('data.xlsx')
# Filter out all rows with no mpi index inside
data_with_mpi = data.dropna(subset=['Country_MPI'], how='all')
# Demonstrate all columns & types
data.dtypes

Country                                             object
Quality Score                                       object
SDGi                                               float64
Life Expectancy                                    float64
HDI                                                float64
Per Capita GDP                                      object
Region                                              object
Income Group                                        object
Population (millions)                               object
Cropland Footprint_p                               float64
Grazing Footprint_p                                float64
Forest Product Footprint_p                         float64
Fish Footprint_p                                   float64
Built up land_p                                    float64
Carbon Footprint_p                                 float64
Total Ecological Footprint (Production)            float64
Cropland Footprint_c                               float

# Visualization
### Overview on World Pollution Situation

In [15]:
# Generate choropleth map
graph1 = px.choropleth(data, locations='Country', locationmode='country names', 
            color='Total Ecological Footprint (Consumption)', scope='world',
            color_continuous_scale='Greys',
            labels={'Total Ecological Footprint (Consumption)': 'SCALE'})

graph1.update_geos(fitbounds="locations", visible=False)

graph1.update_layout(margin={"r":2,"t":2,"l":2,"b":2})

graph1.update_layout(title_text='<br>''Overview on World Total Ecological Footprint Consumption</a>')

# Get JSON specification
graph1json = graph1.to_json('graph1.json')

with open('graph1.json', 'w') as f:
    json.dump(graph1json,f)

# Show interactive plot
graph1.show()

### Relation between Poverty & Ecological Footprints by Region with Footprint Details of Chosen Region

In [6]:
# Define a selection for the y-axis for scatter plot
y_selection_scatter = alt.selection_single(
            name="Choose_scatter",
            fields=["footprints"],
            init={"footprints": "Total Ecological Footprint (Consumption)"},
            bind=alt.binding_select(options=['Total Ecological Footprint (Consumption)', 'Built up land_c', 'Carbon Footprint_c', 
            'Cropland Footprint_c','Fish Footprint_c', 'Forest Product Footprint_c', 'Grazing Footprint_c'])
            )

# Define a selection for the y-axis for line grah
y_selection_line = alt.selection_single(
            name="Choose_line",
            fields=["footprints"],
            init={"footprints": "Total Ecological Footprint (Consumption)"},
            bind=alt.binding_select(options=['Total Ecological Footprint (Consumption)', 'Built up land_c', 'Carbon Footprint_c', 
            'Cropland Footprint_c','Fish Footprint_c', 'Forest Product Footprint_c', 'Grazing Footprint_c'])
            )

# Create the click to select specific region
click = alt.selection_multi(encodings=['color'])

# Create the scatter plot
scatter_plot = alt.Chart(data_with_mpi).mark_circle().encode(
    x=alt.X("Country_MPI:Q", title="MPI Index"),
    y=alt.Y("value:Q", title="Ecological Footprints"),
    color=alt.condition(click, 'Region:N', alt.value('lightgrey')),
    tooltip=["Country", "Region", "Income Group", alt.Tooltip("value:Q", title="Selected Footprint")]
    ).transform_fold(
    fold=['Total Ecological Footprint (Consumption)', 'Built up land_c', 'Carbon Footprint_c', 
    'Cropland Footprint_c','Fish Footprint_c', 'Forest Product Footprint_c', 'Grazing Footprint_c'],
    as_=["footprints", "value"]
    ).transform_filter(
    y_selection_scatter
    ).add_selection(
    y_selection_scatter
    ).add_selection(
    click).interactive()
    
# Create a trend line for each group
trend_line = alt.Chart(data_with_mpi).mark_line().encode(
    x=alt.X("Country_MPI:Q", title="MPI Index"),
    y=alt.Y("value:Q", title="Ecological Footprints"),
    color='Region:N'
    ).transform_fold(
    fold=['Total Ecological Footprint (Consumption)', 'Built up land_c', 'Carbon Footprint_c', 
    'Cropland Footprint_c','Fish Footprint_c', 'Forest Product Footprint_c', 'Grazing Footprint_c'],
    as_=["footprints", "value"]
    ).transform_filter(
    y_selection_line
    ).transform_regression(
    "Country_MPI", "value", groupby=["Region"]
    ).transform_filter(
    click
    ).add_selection(
    y_selection_line
    ).interactive()

# aggregate the scatter plot on line graph
aggregation = scatter_plot + trend_line
aggregation.properties(
    title="Ecological Footprints & MPI",
    width=600,
    height=400
    ).interactive()

# Create a bar chart with details on different types of footprints
bars = alt.Chart(data_with_mpi).mark_bar().encode(
    x=alt.X("value:Q", title="Ecological Footprints"),
    y=alt.Y("footprints:N", title="Footprints Type"),
    color=alt.condition(click, alt.Color('footprints:N'), alt.value('lightgrey'))
    ).transform_fold(fold=[ 'Built up land_c', 'Carbon Footprint_c', 'Cropland Footprint_c','Fish Footprint_c', 
                    'Forest Product Footprint_c', 'Grazing Footprint_c'], as_=["footprints", "value"]
    ).transform_filter(click)

# Concat all graphs and link them with 'click'    
graph2 = alt.vconcat(aggregation, bars).resolve_scale(
    color='independent'
    ).configure_view(
    stroke=None)
    
graph2

graph2.save('graph2.json')